In [ ]:
import sys
sys.path.insert(0, "/home/yymao/Dropbox/Academia/Collaborations/SAGA-local/saga")

In [ ]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import HostCuts as H
from astropy.table import join

In [ ]:
saga = SAGA.QuickStart()

In [ ]:
hosts_with_stats = saga.host_catalog.load(include_stats=True)

In [ ]:
hosts_with_stats['Coverage'] = np.char.add(np.char.add(
    np.where(Query('COVERAGE_SDSS > 0.95').mask(hosts_with_stats), 'S', '-'),
    np.where(Query('COVERAGE_DES_DR1 > 0.95').mask(hosts_with_stats), 'D', '-'),
), np.where(Query('(COVERAGE_DECALS_DR6 > 0.95) | (COVERAGE_DECALS_DR7 > 0.95)').mask(hosts_with_stats), 'L', '-'))

In [ ]:
hosts_with_stats = H.paper2_complete.filter(hosts_with_stats)

In [ ]:
better_p_sat = SAGA.database.FastCsvTable("/home/yymao/Downloads/saga_sat_prob.csv")
if better_p_sat.isfile():
    hosts_with_stats = join(hosts_with_stats, better_p_sat.read()["HOSTID", "sats_missed_pred"], "HOSTID", "left")
    hosts_with_stats["sats_missed_approx"] = hosts_with_stats["sats_missed_pred"]
    del hosts_with_stats["sats_missed_pred"]

In [ ]:
t = hosts_with_stats['COMMON_NAME', 'SAGA_NAME', 'PGC', 'RA', "DEC", 'V_HELIO', 'DIST', 'K_ABS', 'Coverage', 'specs_ours', 'really_need_spec', 'sats_total', 'sats_Mr_limit', "sats_missed_approx"]

In [ ]:
t.sort('RA')

In [ ]:
t['sats_below'] = np.where(t['sats_Mr_limit'] == t['sats_total'], '', np.char.mod('(%d)', t['sats_total']-t['sats_Mr_limit']))
t["sats_missed_approx1"] = t["sats_missed_approx"] 
t['RA'].format = '%.3f'
t['DEC'].format = '%.3f'
t['V_HELIO'].format = '%d'
t['DIST'].format = '%.1f'
t['K_ABS'].format = '%.2f'
t['sats_missed_approx1'].format = '%.3f'

t.rename_column('COMMON_NAME', 'Name')
t.rename_column('SAGA_NAME', 'SAGA Name')
t.rename_column('RA', 'R.A.')
t.rename_column('DEC', 'Dec.')
t.rename_column('V_HELIO', 'HRV')
t.rename_column('DIST', '$d$')
t.rename_column('K_ABS', '$M_{K,o}$')
t.rename_column('specs_ours', r'$N_\text{spec,SAGA}$')
t.rename_column('really_need_spec', r'$N_\text{spec,need}$')
t.rename_column('sats_Mr_limit', r'$N_\text{sat}$')
t.rename_column('sats_below', r'~')
t.rename_column('sats_missed_approx1', r'$N_\text{sat,pred}$')
del t['sats_total'], t['sats_missed_approx']

In [ ]:
print(len(t))

In [ ]:
from io import StringIO
o = StringIO()
t.write(o, format="ascii.aastex")
print(o.getvalue())